# Route Optimization

This notebook demonstrates how to optimize travel routes for multiple users using OpenAI's GPT-4 model. The optimized routes are then visualized on interactive maps using Folium and geocoded using the Geopy library.

**Dataset Overview:**

The CSV file contains the following columns:

- **User**: The name of the user
- **Starting Location**: The starting location of the user
- **Destination**: The destination of the user
- **Waypoints**: The waypoints the user wants to visit

Our goal is to determine the most efficient visiting order for each user to minimize travel time.

### Import Necessary Libraries

We start by importing the essential libraries required for data handling and interaction with the OpenAI API.

- `openai`: To access OpenAI's GPT models for route optimization.
- `pandas`: For data manipulation and analysis.
- `os`: To access environment variables for secure API key handling.

In [75]:
#Import Necessary Libraries:
import openai
import pandas as pd
import os

### Install and Import Visualization Libraries

We install and import the libraries necessary for geocoding addresses and visualizing the routes on a map.

- `folium`: Used for creating interactive map visualizations.
- `geopy`: Utilized for geocoding addresses to geographical coordinates.


In [76]:
# Install visualization libraries
%pip install folium geopy

import folium
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

Note: you may need to restart the kernel to use updated packages.



### Initialize Geolocator

We initialize the `geolocator` using `Nominatim`, which is an open-source geocoding service. The RateLimiter is used to ensure we don't exceed request limits when making multiple geocoding requests.

In [77]:
# Initialize geolocator with Nominatim
geolocator = Nominatim(user_agent="route_optimizer")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

### Load User Routes Data

We read the user routes data from the `user_routes.csv` file into a pandas DataFrame. This data contains all the necessary information for route optimization.

In [86]:
# Read the CSV file into a DataFrame
df = pd.read_csv('user_routes.csv')

# Initialize the OpenAI client
client = openai.OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))


### Define Route Optimization Function
We define the optimize_route function, which utilizes the OpenAI GPT-4 model to optimize the order in which the user should visit each waypoint to minimize travel time.

Function Overview:

Inputs:
- `user_name`: The name of the user.
- `start`: The starting location of the user.
- `destination`: The end location of the user.
- `waypoints`: A list of waypoints the user wants to visit.

Process:
- Constructs a conversation with the GPT model to request an optimized route.
- Sends the request to the OpenAI API.
- Receives and returns the optimized route as a response.

In [87]:
def optimize_route(user_name, start, destination, waypoints):
    messages = [
        {
            "role": "system",
            "content": "You are an assistant helping to optimize travel routes."
        },
        {
            "role": "user",
            "content": f"""
Optimize the visiting order for {user_name} to minimize travel time.

Starting Location:
{start}

Destination:
{destination}

Waypoints:
{', '.join(waypoints)}

Provide the locations in the most efficient visiting order, starting from the starting location and ending at the destination, in the following JSON format:

{{"locations": ["Location1", "Location2", "Location3", ...]}}

**Important:** Only provide the JSON response without any additional text or explanations.
"""
        }
    ]
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=messages,
        max_tokens=2000,
        temperature=0
    )
    optimized_route = response.choices[0].message.content.strip()
    return optimized_route

### Process Each User's Route

We iterate over each user in the DataFrame and call the `optimize_route` function to get the optimized route for that user. The results are stored in a dictionary for later visualization.

In [88]:
#Process each user's route

optimized_routes = {}
for index, row in df.iterrows():
    user = row['User']
    start = row['Starting Location']
    destination = row['Destination']
    waypoints = [wp.strip() for wp in row['Waypoints'].split(';')]
    optimized_route = optimize_route(user, start, destination, waypoints)
    optimized_routes[user] = optimized_route


### Display Optimized Routes

We print out the optimized routes for each user to verify the responses from the GPT model.

In [89]:
for user, route in optimized_routes.items():
    print(f"{user}'s optimized route:\n{route}\n")

Alice's optimized route:
```json
{"locations": ["1600 Amphitheatre Parkway", "Golden Gate Bridge, SF", "Coit Tower, SF", "Lombard Street, SF", "Pier 39, SF", "1 Infinite Loop"]}
```

Bob's optimized route:
{"locations": ["Times Square", "Central Park, NY", "Empire State Building, NY", "One World Trade Center, NY", "Brooklyn Bridge, NY", "Statue of Liberty"]}

Charlie's optimized route:
{"locations": ["Eiffel Tower", "Arc de Triomphe, Paris", "Montmartre, Paris", "Notre-Dame Cathedral, Paris", "Musée d'Orsay, Paris", "Palace of Versailles, Paris", "Louvre Museum"]}

Dana's optimized route:
{"locations": ["Sydney Opera House", "The Rocks, Sydney", "Sydney Harbour Bridge, Sydney", "Darling Harbour, Sydney", "Taronga Zoo, Sydney", "Bondi Beach"]}

Ethan's optimized route:
```json
{"locations": ["Taj Mahal", "Akshardham Temple, Delhi", "Red Fort, Delhi", "Humayun's Tomb, Delhi", "Lotus Temple, Delhi", "Qutub Minar, Delhi", "India Gate"]}
```



In [90]:
optimized_routes

{'Alice': '```json\n{"locations": ["1600 Amphitheatre Parkway", "Golden Gate Bridge, SF", "Coit Tower, SF", "Lombard Street, SF", "Pier 39, SF", "1 Infinite Loop"]}\n```',
 'Bob': '{"locations": ["Times Square", "Central Park, NY", "Empire State Building, NY", "One World Trade Center, NY", "Brooklyn Bridge, NY", "Statue of Liberty"]}',
 'Charlie': '{"locations": ["Eiffel Tower", "Arc de Triomphe, Paris", "Montmartre, Paris", "Notre-Dame Cathedral, Paris", "Musée d\'Orsay, Paris", "Palace of Versailles, Paris", "Louvre Museum"]}',
 'Dana': '{"locations": ["Sydney Opera House", "The Rocks, Sydney", "Sydney Harbour Bridge, Sydney", "Darling Harbour, Sydney", "Taronga Zoo, Sydney", "Bondi Beach"]}',
 'Ethan': '```json\n{"locations": ["Taj Mahal", "Akshardham Temple, Delhi", "Red Fort, Delhi", "Humayun\'s Tomb, Delhi", "Lotus Temple, Delhi", "Qutub Minar, Delhi", "India Gate"]}\n```'}

### Extract Locations from Optimized Routes

We define a helper function extract_locations that parses the optimized route text provided by the GPT model and extracts a list of locations in the order they should be visited.

In [91]:
import re
import json

def extract_locations(route_text):
    try:
        # Remove code fences if present
        json_text = route_text.strip()
        json_text = re.sub(r'^```json', '', json_text)
        json_text = re.sub(r'```$', '', json_text)
        # Parse the JSON string
        data = json.loads(json_text)
        locations = data.get('locations', [])
        return locations
    except json.JSONDecodeError as e:
        print(f"JSON decoding failed: {e}")
        return []

### Geocode Locations and Visualize Routes

For each user's optimized route:

1. Geocode Locations:
   - Convert the location names into geographical coordinates using the geocode function.
   - Handle any locations that fail to geocode by logging and skipping them.

2. Create Interactive Map:
   - Initialize a Folium map centered on the first valid location.
   - Plot the route using a polyline to connect the coordinates.
   - Add markers for each location with color-coded icons:
     - Green: Starting location.
     - Blue: Waypoints.
     - Red: Destination.

3. Save Map to HTML File:
   - Save the interactive map as an HTML file named after the user.

`Note: If no valid coordinates are found for a user's route, a message is displayed, and the map creation is skipped for that user.`

In [92]:
# Geocode locations and visualize routes
for user, route_text in optimized_routes.items():
    print(f"Visualizing {user}'s route...")
    locations = extract_locations(route_text)
    if not locations:
        print(f"No locations found for {user}'s route.")
        continue

    # Geocode locations
    coords = []
    valid_locations = []
    for loc in locations:
        location = geocode(loc)
        if location:
            coords.append((location.latitude, location.longitude))
            valid_locations.append(loc)
        else:
            print(f"Geocoding failed for {loc}")
            # Optionally skip this location or handle it as needed

    if not coords:
        print(f"No valid coordinates for {user}'s route.")
        continue

    # Create a map centered around the first location
    map_center = coords[0]
    m = folium.Map(location=map_center, zoom_start=13)

    # Add markers and lines to the map
    folium.PolyLine(coords, color="blue", weight=2.5, opacity=1).add_to(m)

    for i, (coord, loc) in enumerate(zip(coords, valid_locations)):
        if i == 0:
            color = 'green'  # Starting location
        elif i == len(coords) - 1:
            color = 'red'    # Destination
        else:
            color = 'blue'   # Waypoints
        folium.Marker(
            location=coord,
            popup=f"{i+1}. {loc}",
            icon=folium.Icon(color=color)
        ).add_to(m)

    # Save the map to an HTML file
    map_filename = f"{user}_route_map.html"
    m.save(map_filename)
    print(f"Map saved as {map_filename}\n")

Visualizing Alice's route...
Map saved as Alice_route_map.html

Visualizing Bob's route...
Map saved as Bob_route_map.html

Visualizing Charlie's route...
Geocoding failed for Palace of Versailles, Paris
Map saved as Charlie_route_map.html

Visualizing Dana's route...
Map saved as Dana_route_map.html

Visualizing Ethan's route...
Map saved as Ethan_route_map.html




### Sample Output

After running the code, you will have HTML files for each user:

- `Alice_route_map.html`
- `Bob_route_map.html`
- `Charlie_route_map.html`
- `Dana_route_map.html`
- `Ethan_route_map.html`

**Example of Viewing the Map:**

1. **Open the HTML File:**

   - Locate the file in your directory and open it in a web browser.

2. **Interactive Features:**

   - **Markers:** Click on markers to see the location details.
   - **Zoom and Pan:** Navigate the map to explore the route.

3. **Route Visualization:**

   - **Line Path:** Shows the sequence in which locations should be visited.
   - **Color-coded Markers:**
     - **Green:** Starting location.
     - **Blue:** Waypoints.
     - **Red:** Destination.

Shown below is sample output for Dana's route map:
![Dana's Route Map](Dana_route_map.png)

### Conclusion 

By following these steps and running the provided code, you should be able to generate interactive maps for all users based on the optimized routes. Each map will visually represent the most efficient visiting order of the locations, starting from the green marker and ending at the red marker, with waypoints in between.
